PomodoroAutoTracker

In [1]:
from ipywidgets import VBox, HBox, FloatProgress, ToggleButtons, Text, Button, Tab, Layout
from threading import Timer # start() cancel
from multiprocessing import Process # terminate() start()
from datetime import timedelta, datetime
from time import sleep
from pandas import DataFrame, DatetimeIndex
import qgrid

In [2]:
statuses = ['work','longBreak','shortBreak']
cs = dict(zip(statuses,[0]*3))
cs['pause']=0
state, wait, when = 'work', 5, datetime.now()+timedelta(seconds=5)
ttt, dnt, pp = None, None, None

In [3]:
floatProgress = FloatProgress( description='FloatProgressFloatProgress', style = {'description_width': 'initial'} , layout=Layout(width='40%'))
toggleButtons = ToggleButtons( options=['Start ', 'Pause ', 'Stop '], index=2, description='', 
                                  icons=['play', 'pause', 'eject'], disabled=False, button_style='', layout=Layout(width='40%'))

snoozeButton = Button( description='Add a minute', tooltip='Delay next log', icon='history') #calendar-plus-o
nextButton = Button( description='Finish early', tooltip='Log & skip to next', icon='fast-forward' ) #check
statusText = Text( description='Status', placeholder='CurrentStatus - due in hh:mm | Action', layout=Layout(width='80%'))#style = {'description_width': 'initial'},

# setup times
df_times = DataFrame(data=[24,18,3],index=statuses,columns=['Duration(min)'],dtype=int) #90,60,30
qw_times = qgrid.show_grid(df_times, show_toolbar=False)

# activities
df_todo = DataFrame(data=[['Planning','default task','commit message', True],
                             ['Chores'  ,'Dishes'      ,'hate it!', False]], 
                     columns=['Activity','Task','Comment', 'Success'] )
df_todo.index.set_names('indexName', inplace=True)
qw_todo = qgrid.show_grid(df_todo, show_toolbar=True, column_definitions={'indexName':{'width':37},'Success':{'width':16}})
# log
df_log = DataFrame(data  =[['App','Started','Hello world!', True ]], 
                     index=DatetimeIndex([datetime.now()], name='DatetimeIndex'),
                     columns=['Activity','Task','Comment','Success'] )
qw_log = qgrid.show_grid(df_log, show_toolbar=False, column_definitions={'DatetimeIndex':{'width':40},'Success':{'width':16}})
# sum
df_sum = DataFrame( columns=['Cumulative'], dtype='timedelta64[ns]' )
qw_sum = qgrid.show_grid( df_sum)

# layout
upperbox = VBox([ HBox([ toggleButtons, floatProgress]),
                  HBox([ nextButton   , snoozeButton ]),
                 statusText ])
tab = Tab(children = [qw_todo, qw_log, qw_times, qw_sum ])
tab.set_title(0, 'Activities')
tab.set_title(1, 'Log')
tab.set_title(2, 'TimeSetup')
tab.set_title(3, 'Summary')
VBox( [ upperbox, tab] )

In [4]:
def display_tick(wait, when):
    #print('display_tick', end=';')
    global floatProgress
    delta = when - datetime.now()
    c = 100 * ( 1 - delta.total_seconds() / wait )
    if c < 15:
        floatProgress.bar_style = 'success' #green
    elif c < 50:
        floatProgress.bar_style = '' #blue
    elif c < 70:
        floatProgress.bar_style = 'info' #light blue
    elif c < 85:
        floatProgress.bar_style = 'warning' #orange
    else:
        floatProgress.bar_style = 'danger' #red
    floatProgress.description = state+' '+str(delta).split('.')[0] 
    floatProgress.value = c

def tick_timer( wait, when):
    #print('tick_timer', end=';')
    global ttt
    if when > datetime.now():
        display_tick( wait, when)
    if when+timedelta(seconds=3.0) > datetime.now():
        ttt = Timer(3.0, tick_timer, args=( wait, when)) #Timer(interval, function, args=None, kwargs=None) start, cancel
        ttt.start()
    
def doNext():
    #print('do next', end=';')
    global state, cs, wait
    log()
    oldState=state
    cs[state]+=1
    if state=='work':
        if cs[state]%4 == 0:
            state='longBreak'
        else:
            state='shortBreak'
    elif state=='shortBreak' or 'longBreak':
        state='work'
    else:
        print('wtf!')
        return
    print('%s->%s'%(oldState,state))
    wait=int(qw_times.get_changed_df().loc[state,'Duration(min)'])
    waitFor(wait)
    
def waitFor(wait):
    print('waitFor', wait, end=';')
    global when, dnt #, ttt
    statusText.value = 'state:{}, wait:{}, cs:{}'.format(state, wait, cs)
    when = datetime.now()+timedelta(seconds=wait)
    dnt = Timer(wait, doNext) #, args=(qw_times,)
    dnt.start()
    tick_timer(wait, when)
    #ttt = Timer(2.0, tick_timer, args=(wait, when))
    #ttt.start()

def stopAll():
    print('stopAll', end=';')
    global ttt, dnt
    if dnt!=None and dnt.is_alive():
        dnt.cancel()
        dnt.join()
    if ttt!=None and ttt.is_alive():
        ttt.cancel()
        ttt.join()
    sleep(0.1)
    
def toggleButtonsHandler(event):
    print('toggleButtonsHandler', end=';')
    global wait, when, state
    if event['name'] == 'label':
        #print([ (k,event[k]) for k in event.keys()])
        if event['new'] == 'Start ' and event['old']!='Pause ':
            print('button start')
            doNext()
        elif event['new'] == 'Pause ':
            print('button pause')
            stopAll()
            wait = int((when - datetime.now()).total_seconds())
            log()
        elif event['new'] == 'Start ' and event['old']=='Pause ':
            print('button REstart')
            oldState=state
            state='pause'
            cs[state]+=1
            log()
            state = oldState
            waitFor(wait)
        elif event['new'] == 'Stop ':
            print('button stop')
            stopAll()

def log():
    global qw_log
    sr = qw_todo.get_selected_rows()[0]
    if state=='work':
        qw_log.add_row( [ ('DatetimeIndex',datetime.now()) ]+ list(qw_todo.get_changed_df().loc[sr].items()) )
        qw_todo.edit_cell(sr, 'Success', False)
    else:
        qw_log.add_row( [ ('DatetimeIndex',datetime.now()), ('Activity',state), ('Task',state),('Comment',cs[state]), ('Success',False) ] )

def snoozeButtonHandler(event):
    print('snoozeButtonHandler')
    global wait
    stopAll()
    wait = int((when - datetime.now()).total_seconds() + 60 )
    waitFor(wait)
        
def nextButtonHandler(event):
    print('nextButtonHandler')
    global wait
    stopAll()
    doNext()

nextButton.on_click(nextButtonHandler)
snoozeButton.on_click(snoozeButtonHandler)
toggleButtons.observe(toggleButtonsHandler)

In [5]:
def tabhandler(event):
    global df_sum, qw_sum
    #print(event.keys())
    #print([ (k,event[k]) for k in event.keys()])
    if tab.selected_index == 3:
        print('tabhandler')
        df = qw_log.get_changed_df()
        df['Cumulative'] = df.index.to_series().diff()
        df_sum = DataFrame( df[1:].groupby(['Success','Activity','Task']).Cumulative.sum() )
        qw_sum.df = df_sum
        
tab.observe(tabhandler)

In [6]:
qw_todo.change_selection([0])

In [24]:
df = qw_log.get_changed_df()
df['Cumulative'] = df.index.to_series().diff()
qgrid.show_grid( DataFrame( df[1:].groupby(['Success','Activity','Task']).Cumulative.sum() , columns=['Cumulative'], dtype='timedelta64[ns]' ), show_toolbar=False )

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [29]:
a = df[1:].groupby(['Success','Activity','Task']).Cumulative.sum()
a

Success  Activity    Task                       
False    Chores      asdfas                        0 days 00:02:03.827848
         Planning    default task task task task   0 days 00:00:05.837953
         longBreak   longBreak                     0 days 00:00:20.417678
         shortBreak  shortBreak                    0 days 00:00:27.336437
True     Chores      Dishes                        0 days 00:00:05.195010
         Planning    default task                  0 days 00:00:21.421315
                     default task task             0 days 00:00:09.378551
Name: Cumulative, dtype: timedelta64[ns]

In [44]:
pd.Timedelta.resolution = pd.Timedelta('0 days 00:00:00')
a
#pd.Timedelta.round(a.iloc[0])

Success  Activity    Task                       
False    Chores      asdfas                        0 days 00:02:03.827848
         Planning    default task task task task   0 days 00:00:05.837953
         longBreak   longBreak                     0 days 00:00:20.417678
         shortBreak  shortBreak                    0 days 00:00:27.336437
True     Chores      Dishes                        0 days 00:00:05.195010
         Planning    default task                  0 days 00:00:21.421315
                     default task task             0 days 00:00:09.378551
Name: Cumulative, dtype: timedelta64[ns]

In [43]:
qgrid.show_grid( a )

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [74]:
#a.values.total_seconds() #*1e-9
[ (v.seconds, v.total_seconds()//60, v.seconds%60, v.to_pytimedelta()) for v in a]

[(123, 2.0, 3, datetime.timedelta(seconds=123, microseconds=827848)),
 (5, 0.0, 5, datetime.timedelta(seconds=5, microseconds=837953)),
 (20, 0.0, 20, datetime.timedelta(seconds=20, microseconds=417678)),
 (27, 0.0, 27, datetime.timedelta(seconds=27, microseconds=336437)),
 (5, 0.0, 5, datetime.timedelta(seconds=5, microseconds=195010)),
 (21, 0.0, 21, datetime.timedelta(seconds=21, microseconds=421315)),
 (9, 0.0, 9, datetime.timedelta(seconds=9, microseconds=378551))]

In [69]:
a.ctime()

AttributeError: 'Series' object has no attribute 'ctime'

In [7]:
df = qw_log.get_changed_df()
df['Cumulative'] = df.index.to_series().diff()
df_sum = DataFrame( df[1:].groupby(['Success','Activity','Task']).Cumulative.sum() )
qw_sum.df = df_sum